In [1]:
fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/T5_base_prefix_text2cypher_with_cell_value/predictions_eval_72.72544080604534.json'

In [2]:
schema_fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json'
import json
with open(schema_fpath, 'r', encoding="utf-8") as f:
    schema = json.load(f)
    print(len(schema))

155


In [ ]:
import json

with open(fpath, 'r', encoding="utf-8") as f:
    data = json.load(f)
    print(len(data))
    for each in data:
        # print(each)
        preds = each['prediction']
        preds = preds.replace(" ` ", "`")
        print(f'preds: {preds}')
        mentioned_tags = []
        normalised_db_flag = normalised_tag_flag = normalised_property_flag = False
        tag_span_start_id=tag_span_end_id = 0
        prop_start_id=prop_end_id = 0
        targeted_properties = []

        for id, c in enumerate(preds):
            if c == '`': 
                # the indicator starting postprocessing the predicted query, 
                # including the space sitting ahead of the graph nodes/edges and the case of property names regarding targeted labels/types
                if tag_span_start_id!=0:
                    tag_span_end_id=id
                    print("check it out", preds[tag_span_start_id+1: tag_span_end_id])
                    db, tag = preds[tag_span_start_id+1: tag_span_end_id].split('.')
                    print(db, tag)
                    mentioned_tags.append(tag)
                    alias = tag.lower() #TODO

                    if db in schema:
                        # print(schema[db])
                        lower_cased_tags = [original_tag.lower() for original_tag in schema[db]['tag_names_original']]
                        # 1) check the case sensitivity of tag names
                        if tag.lower() in lower_cased_tags:
                            tag_idx = lower_cased_tags.index(tag.lower())
                            # normalised the existing predicted tag names.
                            preds = preds.replace(preds[tag_span_start_id+1:tag_span_end_id], '{}.{}'.format(db, schema[db]['tag_names_original'][tag_idx]))
                            property_names = schema[db]['property_names']
                            targeted_properties = [ prop[1]  for prop in property_names if prop[0]==tag_idx]
                            print(f'targeted_properties: {targeted_properties}')
                            # 2) check if any targeted property name is predicted and its capitalization. 
                            print(preds)
                    starttag_span_start_id_id=tag_span_end_id=0
                else:
                    tag_span_start_id = id
            if len(mentioned_tags)!=0:
                if id+len(mentioned_tags[-1])+1<=len(preds):
                    if c=='.' and preds[id+len(mentioned_tags[-1])+1]!='`':
                        # the indicator of the span of property name
                        prop_start_id=id+1
                    if c==','and preds[id+len(mentioned_tags[-1])+1]!='`':
                        if preds[id-1]==')':
                            prop_end_id=id-1
                        else:
                            prop_end_id=id
                        mentioned_property = preds[prop_start_id:prop_end_id]
                        print(f'mentioned_property: {mentioned_property}')
                        lower_case_property_names = [prop.lower() for prop in targeted_properties]
                        if mentioned_property.lower() in lower_case_property_names:
                            normalised_mentioned_property = targeted_properties[lower_case_property_names.index(mentioned_property.lower())]
                        preds = preds.replace(preds[prop_start_id:prop_end_id], normalised_mentioned_property)
                        print(f'normalised preds: {preds}')
                        prop_start_id=prop_end_id=0
        # print(mentioned_tags)

In [14]:

from py2neo import Graph
import configparser
config = configparser.ConfigParser()
config.read('/home/22842219/Desktop/phd/SemanticParser4Graph/application/config.ini')
filenames = config["FILENAMES"]

neo4j_uri = filenames['neo4j_uri']
neo4j_user = filenames['neo4j_user']
neo4j_password = filenames['neo4j_password']
graph = Graph(neo4j_uri, auth = (neo4j_user, neo4j_password))

In [15]:
pipeline2_rest='/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/metrics/text2cypher/two_steps_finetune.json'
kaya_t5_prefix_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/kaya/output/T5_base_prefix_text2cypher_with_cell_value-postprocessing_all/predictions_predict.json'
kaya_t5_finetune_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/kaya/output/T5_base_finetune_text2cypher_with_cell_value/predictions_predict.json'
code_t5_finetune_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/CodeT5_base_finetune_text2cypher-predicted_postprocessing_removing_space-normalization/predictions_predict.json'
t5_prefix_post_all= '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/T5_base_prefix_text2cypher_with_cell_value-predicted_postprocessing_removing_space-normalization/predictions_predict.json'

In [16]:
import json
from evaluator import Evaluator, print_scores

for f in [pipeline2_rest, kaya_t5_prefix_post_all, kaya_t5_finetune_post_all, code_t5_finetune_post_all,t5_prefix_post_all ]:
    preds = []
    golds = []
    with open(f, 'r', encoding="utf-8") as f:
        data = json.load(f)
        print(len(data))
        for each in data:
            # print(each)
            pred = each['prediction']
            gold = each['query']
            preds.append(pred)
            golds.append(gold)
    evaluator = Evaluator(graph, 'all')
    results = []
    for predicted, gold in zip(preds, golds):
        results.append(evaluator.evaluate_one(gold, predicted))
    evaluator.finalize()

    print_scores(evaluator.scores, 'all')
    rest =  {
        "per_item": results,
        "total_scores": evaluator.scores,
    }


491
preds: MATCH (singer:`concert_singer.singer`) RETURN count(*)
q_res: [(6,)]
p_res: [(6,)]
sorted results. p_sorted: [(6,)], q_sorted: [(6,)]
preds: MATCH (singer:`concert_singer.singer`) RETURN count(*)
q_res: [(6,)]
p_res: [(6,)]
sorted results. p_sorted: [(6,)], q_sorted: [(6,)]
preds: MATCH (singer:`concert_singer.singer`) RETURN singer.Name,singer.Age ORDER BY singer.Age
q_res: [('Joe Sharp', 'Netherlands', 52), ('John Nizinik', 'France', 43), ('Rose White', 'France', 41), ('Timbaland', 'United States', 32), ('Justin Brown', 'France', 29), ('Tribal King', 'France', 25)]
p_res: [('Tribal King', 25), ('Justin Brown', 29), ('Timbaland', 32), ('Rose White', 41), ('John Nizinik', 43), ('Joe Sharp', 52)]
sorted results. p_sorted: [('Joe Sharp', 52), ('John Nizinik', 43), ('Justin Brown', 29), ('Rose White', 41), ('Timbaland', 32), ('Tribal King', 25)], q_sorted: [('Joe Sharp', 'Netherlands', 52), ('John Nizinik', 'France', 43), ('Justin Brown', 'France', 29), ('Rose White', 'France',

post processed predicted by removing space: MATCH (singer:`concert_singer.singer`) RETURN count(*)
gold query: MATCH (singer:`concert_singer.singer`)
RETURN count(*)
preds: MATCH (singer:`concert_singer.singer`) RETURN count(*)
heyyy ['concert_singer', 'singer']
finally:  MATCH (singer:`concert_singer.singer`) RETURN count(*)
normalised_predicted: MATCH (singer:`concert_singer.singer`) RETURN count(*)
self.scores[exec]: 0
q_res: [(6,)]
p_res: [(6,)]
sorted results. p_sorted: [(6,)], q_sorted: [(6,)]
hyyyyyyy check it out: True
q_res: [(6,)]
p_res: [(6,)]
sorted results. p_sorted: [(6,)], q_sorted: [(6,)]
post processed predicted by removing space: MATCH (singer:`concert_singer.singer`) RETURN count(*)
gold query: MATCH (singer:`concert_singer.singer`)
RETURN count(*)
preds: MATCH (singer:`concert_singer.singer`) RETURN count(*)
heyyy ['concert_singer', 'singer']
finally:  MATCH (singer:`concert_singer.singer`) RETURN count(*)
normalised_predicted: MATCH (singer:`concert_singer.singer`)